# DataSet:

In [29]:
Train_data = [['Green',3,'Mango'],['Yellow',3,'Mango'],['Red',1,'Grape'],['Red',1,'Grape'],['Yellow',3,'Lemon'] ] 

In [30]:
# Columns labls
Headers = ["Color","Diameter","Label"]

In [31]:
# unique_values: find the unique values for a column in data

def uniques_values(rows,col):
    return set([row[col] for row in rows])

In [32]:
uniques_values(Train_data,0)

{'Green', 'Red', 'Yellow'}

In [33]:
uniques_values(Train_data,1)

{1, 3}

In [34]:
# Class_count: count the number of each type in the data.

def class_count(rows):
    
    count={}
    for row in rows:
        label=row[-1]
        if label not in count:
            count[label]=0
            
        count[label]+=1
    
    
    return count

In [35]:
class_count(Train_data)

{'Mango': 2, 'Grape': 2, 'Lemon': 1}

In [36]:
# is_numeric: test if a value is numeric.

def is_numeric(val):
    return isinstance(val,int) or isinstance(val, float)

In [37]:
is_numeric(7)

True

In [38]:
is_numeric('7')

False

In [39]:
class Question:
    """ A question is used to partition a dataset.
    
        this class record a column number and column value. 
        the match method is used to compare a feature value in an exemple to the feature 
        value stored in the question."""
    
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def match(self, example):
        # compare the example value with the value in the question.
        
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        
        else :
            return val == self.value
        
    def __repr__(self):
        # method to print the qustion
        
        condition = "=="
        
        if is_numeric(self.value):
            condition = ">="
            
        return "Is %s %s %s ?" %(Headers[self.column],condition,str(self.value))
        

In [40]:
# Partition of dataset
#exemple: True_rows,False_rows=Partition(Train_data,Question(0,'Red'))


def Partition(rows, question):
    """ for each row in data, 
    chek if it matches the question, add it to true rows, else add it to false rows"""
    
    True_rows, False_rows= [], []
    
    for row in rows:
        if question.match(row):
            True_rows.append(row)
            
        else:
            False_rows.append(row)
            
    return True_rows, False_rows
    
    
    

In [41]:
# gini: Calculate the gini impurity for a list of rows

def gini(rows):
    
    counts = class_count(rows)
    impurity=1
    
    for lbl in counts:
        prob_of_lbl = counts[lbl]/float(len(rows))
        impurity -= prob_of_lbl**2
        
    return impurity

In [52]:
# information gain

def info_gain(left, right, current_uncertainty):
    
    p= float(len(left)/(len(left)+len(right)))
    return current_uncertainty - p*gini(left) -(1-p)*gini(right)
    

In [43]:
# find the best question to ask.

def find_the_best_split(rows):
    
    best_gain = 0
    best_question =None
    current_uncertainty = gini(rows)
    n_featurs = len(rows[0])-1
    
    
    for i in range(n_featurs):
        
        values = set([row[i] for row in rows])
        
        for val in values:
            
            question = Question(i,val)
            
            true_rows, false_rows = Partition(rows, question)
            
            if len(true_rows)==0 or len(false_rows)==0:
                continue
                
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            
            if gain>=best_gain:
                best_gain=gain
                best_question = question
                
            
            
    return best_gain, best_question
    

In [44]:

class Leaf:
    
    def __init__(self, rows):
        self.predictions = class_count(rows)

In [56]:

class Decision_Node:
    
    
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        


In [54]:


def build_tree(rows):
    
    # partitioning the data on each of the unique attribute.
    # calculate the information gain
    # return the question that produces the highest gain
    
    gain, question = find_the_best_split(rows)
    
    if gain == 0:
        return Leaf(rows)
    
    
    true_rows, false_rows = Partition(rows, question)
    
    
    true_branch = build_tree(true_rows)
    
    
    false_branch = build_tree(false_rows)
    
    
    return Decision_Node(question, true_branch, false_branch)
    
    

In [47]:
def print_tree(node, spacing=""):
    
    if isinstance(node,Leaf):
        print(spacing + "Predict",node.predictions)
        return
    
    print(spacing + str(node.question))
    
    print(spacing + "--> True")
    
    print_tree(node.true_branch, spacing+ " ")
    
    
    print(spacing + "--> False")
    
    print_tree(node.false_branch, spacing+ " ")
        

In [48]:
def classify(row, node):
    
    if isinstance(node,Leaf):
        return node.predictions
    
    if node.question.match(row):
        return classify(row, node.true_branch)
    
    else:
        return classify(row, node.false_branch)
    

    

In [58]:
def print_leaf(counts):
    total= sum(counts.values())*1.0
    
    probs= {}
    for lbl in counts.keys():
        probs[lbl]= str(int(counts[lbl]/total*100))+"%"
        
    return probs


In [59]:
if __name__ == '__main__':
    my_tree = build_tree(Train_data)
    
    print_tree(my_tree)
    
    
    testing_data= [
        ['Green',3,'Apple'],
        ['Yellow',4,'Apple'],
        ['Red',2,'Grape'],
        ['Red',1,'Grape'],
        ['Yellow',3,'Lemon'],
    ]
    
    
    for row in testing_data:
        print("Actual: %s. Predicted: %s" %
             (row[-1],print_leaf(classify(row, my_tree))))
    
    

Is Diameter >= 3 ?
--> True
 Is Color == Yellow ?
 --> True
  Predict {'Mango': 1, 'Lemon': 1}
 --> False
  Predict {'Mango': 1}
--> False
 Predict {'Grape': 2}
Actual: Apple. Predicted: {'Mango': '100%'}
Actual: Apple. Predicted: {'Mango': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Mango': '50%', 'Lemon': '50%'}
